In [27]:
import pandas as pd
import os
from SPARQLWrapper import SPARQLWrapper, JSON

In [28]:
folderlink = '..//data//'
input_folder = 'input//'
folder_output = 'output//'
ddrc_file = 'parent_data.csv'
input_file = os.path.join(folderlink+folder_output, ddrc_file)


In [29]:
df = pd.read_csv(input_file, sep=';', encoding='utf-8')

In [30]:
sparql_endpoint = "http://data.bibliotheken.nl/sparql"


In [ ]:
sparql_query = """
select distinct ?type where {
[]a ?type.
}
"""

In [31]:
sparql_query = """
select ?publicatie where {
  ?publicatie schema:mainEntityOfPage ?beschrijving .
  ?beschrijving schema:isPartOf <http://data.bibliotheken.nl/id/dataset/stcn> .
} limit 100
"""


In [41]:
sparql_query = """
select * where {
?publicatie schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/nbt> .
?publicatie schema:author/owl:sameAs/schema:gender <http://schema.org/Female> .
}
"""

In [42]:
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

In [43]:
bindings = results["results"]["bindings"]
data = []

for item in bindings:
    row = {}
    for key in item:
        row[key] = item[key]["value"]
    data.append(row)

df = pd.DataFrame(data)

In [44]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [46]:
df.head(500)

,publicatie
0,http://data.bibliotheken.nl/id/nbt/p782098037
1,http://data.bibliotheken.nl/id/nbt/p782974678
2,http://data.bibliotheken.nl/id/nbt/p801849292
3,http://data.bibliotheken.nl/id/nbt/p801976669
4,http://data.bibliotheken.nl/id/nbt/p802522076
5,http://data.bibliotheken.nl/id/nbt/p803146531
6,http://data.bibliotheken.nl/id/nbt/p803171560
7,http://data.bibliotheken.nl/id/nbt/p803433573
8,http://data.bibliotheken.nl/id/nbt/p810465086
9,http://data.bibliotheken.nl/id/nbt/p811584836
